<a href="https://colab.research.google.com/github/hoainv99/classify_comment/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
#hyper parameters
input_size=100
hidden_size=300
output_size=2
num_epochs=50
learning_rate=0.001
batch_size=16


In [0]:
data=np.load('/content/drive/My Drive/best_data.npy')
label=np.load('/content/drive/My Drive/best_label.npy')

In [0]:
lb = []
for i in label:
  if i==0 :
    lb.append([1,0])
  else :
    lb.append([0,1])

In [0]:
x_train,x_test,y_train,y_test=train_test_split(data,lb,test_size=0.33,random_state=42)

In [0]:
type(y_train)

list

In [0]:
class Custom_Dataset(Dataset):
  def __init__(self, data, label):
    self.data = data
    self.label = label
  def __getitem__(self, index):
    X = torch.from_numpy(self.data[index]).to(device).float()
    y = torch.from_numpy(np.array(self.label[index])).to(device).float()
    return torch.squeeze(X),torch.squeeze(y) 
  def __len__(self):
    return len(self.data)

In [0]:
train_set = Custom_Dataset(x_train,y_train)
val_set = Custom_Dataset(x_test,y_test)

train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

In [0]:
x,y = next(iter(train_dataloader))

In [0]:
x.shape,y.shape

(torch.Size([16, 100]), torch.Size([16, 2]))

In [0]:
class NeuralNetword(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(NeuralNetword,self).__init__()
    self.fc1 = nn.Linear(input_size,hidden_size)
    self.fc2 = nn.Linear(hidden_size,hidden_size)
    self.fc3 = nn.Linear(hidden_size,output_size)

  def forward(self,input):
    out = self.fc1(input)
    out = torch.relu(out)
    out = self.fc2(out)
    out = torch.relu(out)
    out = self.fc3(out)
    out = torch.sigmoid(out)
    return out

In [0]:
model=NeuralNetword(input_size,hidden_size,output_size)
model.cuda()
criterion=nn.BCELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)     

AttributeError: ignored

In [0]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [0]:
import tqdm

In [0]:
loss_hist_train = Averager()
loss_hist_val = Averager()
for epoch in range(num_epochs):
    loss_hist_train.reset()
    total_train = total_val = 0
    correct_train = correct_val = 0
    for batch_idx,(images, labels) in enumerate(train_dataloader):
       
        outputs = model(images)
        loss_train = criterion(outputs,labels)

        loss_hist_train.send(loss_train)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f"Iteration #{batch_idx} loss: {loss_train.item()}")
        _,predicted = torch.max(outputs.data,1)
        _,labels = torch.max(labels.data,1)
        total_train += labels.size(0)
        correct_train +=(predicted == labels).float().sum()
    
    for batch_idx,(images, labels) in enumerate(test_dataloader):   
        outputs = model(images)

        loss_val = criterion(outputs,labels)

        loss_hist_val.send(loss_val)

        if batch_idx % 100 == 0:
            print(f"Iteration #{batch_idx} loss: {loss_val.item()}")
        _,predicted = torch.max(outputs.data,1)
        _,labels = torch.max(labels.data,1)
        total_val += labels.size(0)
        correct_val +=(predicted == labels).float().sum()
    lr_scheduler.step()
    print(f"epoch #{epoch} train_loss: {loss_hist_train.value} val_loss: {loss_hist_val.value} acc_train: {100* correct_train / total_train} acc_val:{100* correct_val / total_val }")   
    
    

Iteration #0 loss: 0.6923997402191162
Iteration #100 loss: 0.34957486391067505
Iteration #200 loss: 0.24698050320148468
Iteration #300 loss: 0.3581673502922058
Iteration #400 loss: 0.29601508378982544
Iteration #500 loss: 0.10175138711929321
Iteration #600 loss: 0.29748088121414185
Iteration #0 loss: 0.3314845561981201
Iteration #100 loss: 0.18654516339302063
Iteration #200 loss: 0.29709964990615845
Iteration #300 loss: 0.3870135247707367
epoch #0 train_loss: 0.36797937750816345 val_loss: 0.32788798213005066 acc_train: 83.825439453125 acc_val:85.63619232177734
Iteration #0 loss: 0.34065449237823486
Iteration #100 loss: 0.2831583619117737
Iteration #200 loss: 0.16330185532569885
Iteration #300 loss: 0.2930181622505188
Iteration #400 loss: 0.18316048383712769
Iteration #500 loss: 0.5234241485595703
Iteration #600 loss: 0.15218569338321686
Iteration #0 loss: 0.295199990272522
Iteration #100 loss: 0.3697766661643982
Iteration #200 loss: 0.6169006824493408
Iteration #300 loss: 0.16859325766